# Inference Demo - Driver Drowsiness Detection

This notebook demonstrates how to load a trained model and run inference on sample images.


In [ ]:
import sys
from pathlib import Path
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Add project root to path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from src.models.cnn_model import load_model, predict, preprocess_image
from src.config.settings import MODEL_PATH

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


## 1. Load Trained Model


In [ ]:
# Load model
model = load_model(MODEL_PATH, device=device)
print("Model loaded successfully!")


## 2. Run Inference on Sample Images


In [ ]:
# Get sample images from both classes
data_dir = project_root / "Data"
drowsy_dir = data_dir / "Drowsy"
non_drowsy_dir = data_dir / "Non Drowsy"

drowsy_images = list(drowsy_dir.glob("*.png"))[:5]
non_drowsy_images = list(non_drowsy_dir.glob("*.png"))[:5]

# Run inference
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Inference Results', fontsize=16, fontweight='bold')

for i, img_path in enumerate(drowsy_images):
    img = Image.open(img_path).convert('RGB')
    result = predict(model, img, device)
    
    ax = axes[0, i]
    ax.imshow(img)
    color = 'red' if result['prediction'] == 'drowsy' else 'green'
    ax.set_title(f"{result['prediction']}\n({result['confidence']:.2f})", 
                 color=color, fontweight='bold')
    ax.axis('off')

for i, img_path in enumerate(non_drowsy_images):
    img = Image.open(img_path).convert('RGB')
    result = predict(model, img, device)
    
    ax = axes[1, i]
    ax.imshow(img)
    color = 'red' if result['prediction'] == 'drowsy' else 'green'
    ax.set_title(f"{result['prediction']}\n({result['confidence']:.2f})", 
                 color=color, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()


## 3. Detailed Prediction Example


In [ ]:
# Example with detailed output
sample_img = Image.open(np.random.choice(drowsy_images)).convert('RGB')
result = predict(model, sample_img, device)

print("Prediction Details:")
print(f"  Predicted class: {result['prediction']}")
print(f"  Confidence: {result['confidence']:.4f}")
print(f"  Probabilities:")
for class_name, prob in result['probabilities'].items():
    print(f"    {class_name}: {prob:.4f}")

plt.figure(figsize=(6, 6))
plt.imshow(sample_img)
plt.title(f"Prediction: {result['prediction']} (Confidence: {result['confidence']:.2f})", 
          fontsize=14, fontweight='bold',
          color='red' if result['prediction'] == 'drowsy' else 'green')
plt.axis('off')
plt.tight_layout()
plt.show()
